# Creating and Valuing a CDS Index

Example CDS Index Valuation

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

In [3]:
value_dt = Date(7, 2, 2006)
settle_dt = value_dt.add_weekdays(0)
print(settle_dt)

07-FEB-2006


## Build Ibor Curve

Build the Ibor curve to value cash flows

In [4]:
depos = []
dc_type = DayCountTypes.THIRTY_E_360_ISDA
fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap_type = SwapTypes.PAY
swap1 = IborSwap(settle_dt,"1Y",swap_type,0.0502,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"2Y",swap_type,0.0502,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"3Y",swap_type,0.0501,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"4Y",swap_type,0.0502,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"5Y",swap_type,0.0501,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

## Build Index Curve

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

In [5]:
step_in_dt = value_dt

In [6]:
index_cpn = 0.0048375
maturity_dt = Date(20, 6, 2010)

cds_mkt_contracts = []
cds = CDS(value_dt, maturity_dt, index_cpn)
cds_mkt_contracts.append(cds)

recovery_rate = 0.40
index_curve = CDSCurve(value_dt, cds_mkt_contracts,libor_curve,recovery_rate)

## Create CDS Index and Perform Valuation

The CDS Index is treated as a CDS with the cpn equal to the CDS Index cpn

In [7]:
cdsRecovery = 0.40
notional = 10.0 * ONE_MILLION
long_protection = True
index_cpn = 0.004

cdsIndexContract = CDS(step_in_dt, maturity_dt, index_cpn, notional, long_protection)

In [8]:
spd = cdsIndexContract.par_spread(value_dt, index_curve, cdsRecovery) * 10000
print("PAR SPREAD", spd)

PAR SPREAD 48.37497278598326


In [9]:
v = cdsIndexContract.value(value_dt, index_curve, cdsRecovery)
print("DIRTY VALUE", v['dirty_pv'])
print("CLEAN VALUE", v['clean_pv'])

DIRTY VALUE 27193.705780363234
CLEAN VALUE 32638.150224807672


In [10]:
p = cdsIndexContract.clean_price(value_dt, index_curve, cdsRecovery)
print("CLEAN PRICE", p)

CLEAN PRICE 99.67361849775192


In [ ]:
accrued_days = cdsIndexContract.accrued_days(value_dt)
print("ACCRUED DAYS", accrued_days)

accrued_interest = cdsIndexContract.accrued_interest(value_dt)
print("ACCRUED COUPON", accrued_interest)

ACCRUED DAYS 49.0
ACCRUED COUPON -5444.444444444444


In [12]:
prot_pv = cdsIndexContract.prot_leg_pv(value_dt, index_curve, cdsRecovery)
print("PROTECTION LEG PV", prot_pv)

PROTECTION LEG PV 188522.35932663237


In [13]:
premPV = cdsIndexContract.premium_leg_pv(value_dt, index_curve, cdsRecovery)
print("PREMIUM LEG PV", premPV)

PREMIUM LEG PV 161328.65354626914


In [14]:
rpv01 = cdsIndexContract.risky_pv01(value_dt, index_curve)
print("DIRTY RPV01", rpv01['dirty_rpv01'])
print("CLEAN RPV01", rpv01['clean_rpv01'])

DIRTY RPV01 4.033216338656729
CLEAN RPV01 3.8971052275456177


Copyright (c) 2020 Dominic O'Kane